In [1]:
# custom_loss.py
import torch
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss


In [2]:
def custom_loss(preds, targets):
    # стандартный лосс для bbox/obj/dfl
    loss_items = model.trainer.criterion(preds, targets)

    # классификация
    cls_preds = preds[0].cls
    targets_dict = model.trainer.build_targets(preds, targets)[0]
    cls_targets = targets_dict["cls"].to(cls_preds.device)

    focal_cls_loss = focal_loss(cls_preds, cls_targets)

    # финальный лосс
    total_loss = (
        loss_items["box"] +
        loss_items["obj"] +
        loss_items.get("dfl", 0) +
        focal_cls_loss
    )
    return total_loss


In [3]:
import optuna
from ultralytics import YOLO
import os
import shutil
import torch
import yaml
import numpy as np

# === Настройки ===
DATASET_PATH = "dataset.yaml"  # путь к датасету YOLO (train/val/test)
MODEL = "yolov8n.pt"           # маленькая модель для быстроты
N_TRIALS = 50                   # для теста можно 5-10

# --- функция подсчёта объектов для alpha ---
def count_classes(data_yaml: str):
    with open(data_yaml, "r") as f:
        data = yaml.safe_load(f)
    names = data["names"]
    num_classes = len(names)
    root = data["path"]
    label_dirs = [
        os.path.join(root, "labels", "train"),
        os.path.join(root, "labels", "val")
    ]
    counts = np.zeros(num_classes, dtype=np.int64)
    for label_dir in label_dirs:
        if not os.path.exists(label_dir):
            continue
        for file in os.listdir(label_dir):
            if file.endswith(".txt"):
                path = os.path.join(label_dir, file)
                with open(path, "r") as f:
                    for line in f:
                        if line.strip() == "":
                            continue
                        cls_id = int(line.split()[0])
                        counts[cls_id] += 1
    return names, counts

names, counts = count_classes(DATASET_PATH)
inv = 1.0 / (counts + 1e-6)
alpha = torch.tensor(inv / inv.sum(), dtype=torch.float32)

# === Функция обучения одной эпохи с Optuna ===
def objective(trial):
    # Гиперпараметры для подбора
    lr = trial.suggest_float("lr0", 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.6, 0.98)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    augment = trial.suggest_categorical("augment", [True, False])
    fl_gamma = trial.suggest_float("fl_gamma", 0.0, 3.0)  # для focal loss

    # Временная папка для результатов
    exp_name = f"trial_{trial.number}"
    save_dir = os.path.join("runs", "optuna", exp_name)
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)

    # Загружаем YOLO
    model = YOLO(MODEL)

    # --- создаём FocalLoss с текущим gamma ---
    focal_loss = FocalLoss(alpha=alpha, gamma=fl_gamma)

    # --- callback для подмены cls_loss ---
    def custom_loss(preds, targets):
        loss_items = model.trainer.criterion(preds, targets)
        cls_preds = preds[0].cls
        targets_dict = model.trainer.build_targets(preds, targets)[0]
        cls_targets = targets_dict["cls"].to(cls_preds.device)
        focal_cls_loss = focal_loss(cls_preds, cls_targets)
        total_loss = (
            loss_items["box"] +
            loss_items["obj"] +
            loss_items.get("dfl", 0) +
            focal_cls_loss
        )
        return total_loss

    def on_train_start(trainer):
        trainer.loss = custom_loss

    model.add_callback("on_train_start", on_train_start)

    # Тренировка на 1-2 эпохи для Optuna
    results = model.train(
        data=DATASET_PATH,
        epochs=2,
        imgsz=640,
        batch=16,
        lr0=lr,
        momentum=momentum,
        weight_decay=weight_decay,
        augment=augment,
        project="optuna_runs",
        name=exp_name,
        exist_ok=True,
        verbose=False,
        workers=2,
    )

    # Используем метрику mAP50 для оценки
    return results.results_dict["metrics/mAP50(B)"]

# === Optuna study ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("Лучшие параметры:", study.best_params)
print("Лучший mAP50:", study.best_value)


s:\hacatons\tools\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-01 22:51:20,797] A new study created in memory with name: no-name-ec7dc7ef-2bf9-44a8-a29d-5743c5db39ef


Transferred 319/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.12.6 torch-2.10.0.dev20250926+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001067835247751057, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.904

[I 2025-10-01 22:52:41,023] Trial 0 finished with value: 0.9156597874480498 and parameters: {'lr0': 0.0001067835247751057, 'momentum': 0.9047649973523264, 'weight_decay': 8.948754616291133e-05, 'augment': False, 'fl_gamma': 0.921728986571275}. Best is trial 0 with value: 0.9156597874480498.


Transferred 319/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.12.6 torch-2.10.0.dev20250926+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=4.3949831543548444e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.947

[I 2025-10-01 22:53:53,877] Trial 1 finished with value: 0.9116748428051262 and parameters: {'lr0': 4.3949831543548444e-05, 'momentum': 0.9473134518895222, 'weight_decay': 0.0018640315890027892, 'augment': True, 'fl_gamma': 0.7210184131086543}. Best is trial 0 with value: 0.9156597874480498.


...

Transferred 319/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.12.6 torch-2.10.0.dev20250926+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003597928212382573, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.7250

[I 2025-10-01 23:55:56,479] Trial 48 finished with value: 0.9153806057185689 and parameters: {'lr0': 0.003597928212382573, 'momentum': 0.7250176364500128, 'weight_decay': 0.0001823877269303519, 'augment': False, 'fl_gamma': 1.3689956155627638}. Best is trial 23 with value: 0.9179732764342421.


Transferred 319/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.12.6 torch-2.10.0.dev20250926+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0015255173062021414, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.683

[I 2025-10-01 23:57:15,954] Trial 49 finished with value: 0.9135026124830024 and parameters: {'lr0': 0.0015255173062021414, 'momentum': 0.6834230014388477, 'weight_decay': 0.0032784424519281493, 'augment': False, 'fl_gamma': 2.497425786287253}. Best is trial 23 with value: 0.9179732764342421.


Лучшие параметры: {'lr0': 0.00044372638023533847, 'momentum': 0.8792977803712093, 'weight_decay': 0.0015776452546715513, 'augment': False, 'fl_gamma': 2.2294182134740304}
Лучший mAP50: 0.9179732764342421
